# Assignment 1: Performance Metrics & First Week Flop/s

This assignment has some questions that you need to answer with text, and some code that you need to write.

You should put all of you textual answers in this notebook: `Insert->Insert Cell Below` to create a new cell below
the question, and `Cell->Cell Type->Markdown` to make it a cell for entering text.

You will test your code on the compute nodes of pace-ice, and that it also where we will evaluate it.
Please complete the text portions when you are logged into a head node working locally, and leave the compute nodes for when you actually need them.

## Performance Metrics

In class we talked about the _strong-scaling efficiency_ of a parallel algorithm / machine pair: $H_f(P) = T_f(1) / (P T_f(P))$.

We then talked about the _weak-scaling efficiency_ of algorithm $f$ that can be applied to different problem sizes $N$: $E_f(N,P) = T_f(N/P,1) / T_f(N,P)$.

The question came up of how they are related to each other.

First, the notion of strong scaling doesn't have a concept of problem size, so let's add it: let's define

$$H_f(N,P) = T_f(N,1) / (P T_f(N,P)).$$

This is simply strong-scaling efficiency for each problem instance individually.

**Question 1 (1 pt):** Show that the relative order of strong and weak scaling efficiency can be related to the efficiency of the serial algorithm $T_f(N,1)$, that is, whether it exhibits superlinear or sublinear behavior.

## PACE-ICE

**Head node exercise 1 (1 pt):** What command should you run from a head node to see a list of all the compute nodes and their availability?

In [1]:
# <-- Put your command there

Try it out: open up this notebook on a head node and compare the list you get to the [orientation slides](http://pace.gatech.edu/sites/default/files/pace-ice_orientation_0.pdf).  You'll see that it has grown, and they haven't updated the orientation slides.  We'll just have to find out what all these new nodes are for ourselves.

**Head node exercise 2 (1 pt):** For the next questions, I need you to log in to compute nodes to find out about them, but I need to be able to specify which type of compute nodes you are accessing.

For each of the types of nodes that you see in the list of resources you've created, give me the `qsub` command to start an interactive job on that type of node, with the following requirements:

* The job should give you exclusive access to one node and all its cores and devices.
* The job should let you pop open an X window (like a notebook) if you want to.
* The job should begin in the CSE6230 directory.
* The job should end after 30 minutes.

In [2]:
# put one qsub command in this cell, and duplicate the cell for the others

## What have we got to work with?

Now, we need to switch from a notebook running on the head node to one running on a compue node, so `File->Save and Checkpoint` this notebook and `File->Close and Halt` it.  (Now would also be a good time to `git add` and `git commit` changes to this file.)  Use one of your ineractive job scripts to connect to a compute node and run the notebook there.
See you on the other side!

---

Okay, you're running on the compute node.

**Compute node exercise 1 (1 pt):** 